- First part of the video was about optimizing the initial loss value of the neural network. He reduced this by (1) initializing our final bias b2 to 0, reducing W2 down by multiplying it by 0.1.
    - The intuition here is that we want our final output (logits) to be as close to zero as possible. Since there are 27 characters that our neural network can predict, it makes sense that they should have even probability split between them at initialization. If all of our logits have the same value, we get this equal distribution. The easiest way to give them all the same value is to make them all close to zero.
    - Since logits = h @ W2 + b2, reducing W2 and removing b2 help us with this.

- There is a second problem todo with h, which is our tanh'd hidden layer. Most of the values are -1 or 1, because tanh is very "active"; basically
  - This is a problem because the gradient of these tanh nodes becomes close to zero, which bungs up our loss function.
  - To fix this, similar to above, we want hpreact to be close to zero; we can squash the initial weights to do this.
  - Interesting: If you multiply a vector like W1 by some fraction, it appears to act as the standard deviation.

### Batch normalization
- Relevant paper https://arxiv.org/abs/1502.03167
- We have hidden states hpreact (hidden layer pre-activation), and we don't want these to be too small or large because that means tanh squashes them.
  - We want these to be roughly Gaussian; 0 mean and standard deviation of 1 at initialization.
- The batch normalization paper just asks: Why not just take the hidden states and initialize them to be Gaussian?
  - There is a fairly simple formula for doing this.
- One thing that makes this a little more complex is that we only want these to be Gaussian at initialization time, after that, we want the weights to be able to move as needed as we train.
  - To handle this, we use a 'scale and shift' pattern, where we create a batch normalization gain (bngain) and a batch normalization bias (bnbias), initialized as ones and zeroes respectively, that will then multiply (bngain) and add to (bnbias) hpreact. During training, bngain and bnbias are backpropagated so they are also optimized.
- Batch normalization acts as a regularizer (I didn't really understand the reasoning here lol); examples in a batch are coupled together
- Another problem batch normalization creates: When a neural network is actually deployed, we want to be able to feed in 1 input and get one output, but batch normalization now requires us to feed in batches.
  - The paper proposes a solution in which we have a step after training, that calculates and sets the batch normalization mean and standard deviation a single time. We call this "calibrating". You then use these at test time / inference time instead of calculating them manually.
  - You can estimate the mean and SD during training by calculating a running mean and SD, which alleviates the need for an extra step. The way this was done in the example was pretty janky - you basically just update it a tiny bit at each iteration. 